# Лабораторная работа № 5.
Нахождение собственных чисел и собственных векторов матрицы  
Вариант 9

In [2]:
import numpy as np



np.set_printoptions(2)

$a_{ij}=e^{-(i+j)\alpha}$  
$i,j=1,...,n$  
$0\le \alpha\le max(i, j)$



In [3]:
def make_matrix(n):
    low = 10 ** -6
    rng = np.random.default_rng()
    A = np.zeros((n, n))
    for i in range(n):
        for j in range(i, n):
            high = 1 / max(i + 1, j + 1)
            alpha = rng.uniform(low, high)
            A[j, i] = A[i, j] = np.exp(-(i + j + 2) * alpha)

    return A

Метод скалярных произведений (частичная проблема поиска собственных значений)

In [7]:
def scalar_product_method(A, eps=1e-10, max_iterations=10**6):
    assert A.ndim == 2
    assert A.shape[0] == A.shape[1]

    n = A.shape[0]
    iterations = 0

    y_prev = np.random.rand(n)
    y1_prev = y_prev.copy()

    y_prev /= np.linalg.norm(y_prev)
    y1_prev /= np.linalg.norm(y1_prev)

    eigen_value = np.dot(y_prev, y1_prev) / np.dot(y_prev, y1_prev)
    prev_eigen_value = eigen_value

    while iterations < max_iterations:
        iterations += 1

        y_new = A @ y_prev
        y1_new = A.T @ y1_prev

        eigen_value = np.dot(y_new, y1_new) / np.dot(y_prev, y1_new)

        if abs(eigen_value - prev_eigen_value) < eps:
            break

        prev_eigen_value = eigen_value
        y_prev, y1_prev = y_new, y1_new

    eigen_vector = y_new / np.linalg.norm(y_new)

    return eigen_value, eigen_vector


Степенной метод (частичная проблема поиска собственных значений)  
Предполагается, что матрица А действительная с положительными коэффициентами

In [6]:
def power_method(A, eps=1e-10, max_iterations=10**6):
    assert A.ndim == 2
    assert A.shape[0] == A.shape[1]

    iterations = 0
    n = A.shape[0]

    y_prev = np.random.rand(n)
    y_prev /= np.linalg.norm(y_prev)
    y_new = y_prev.copy()

    while (iterations < max_iterations):
        iterations += 1

        y_new = A @ y_prev
        y_new /= np.linalg.norm(y_new)

        if np.linalg.norm(y_new - y_prev) < eps:
            break

        y_prev = y_new

    max_eigen_value = np.dot(A @ y_new, y_new)
    eigen_vector = y_new / np.linalg.norm(y_new)

    return max_eigen_value, eigen_vector


Метод Якоби, метод вращений (полная проблема поиска собственных значений)  
Предполагается, что матрица А вещественная и симметричная

In [4]:
def measure(A):
    s = 0
    n = A.shape[0]

    for i in range(n):
        for j in range(n):
            if i == j:
                continue
            s += A[i, j] ** 2

    return s


def max_pos(A):
    n = A.shape[0]
    m = np.finfo("float64").min
    pos = (0, 0)

    for i in range(n):
        for j in range(n):
            if i == j:
                continue
            if abs(A[i, j]) > m:
                m = abs(A[i, j])
                pos = (i, j)

    return pos


def jacobi_method(A, eps=10**-10, max_iterations=10**6):
    assert A.ndim == 2
    assert A.shape[0] == A.shape[1]

    iterations = 0
    n = A.shape[0]
    Q = np.eye(n)

    while (measure(A) >= eps) and (iterations <= max_iterations):
        i, j = max_pos(A)

        if A[i, i] != A[j, j]:
            angle = np.arctan(2 * A[i, j] / (A[i, i] - A[j, j])) / 2
        else:
            angle = np.pi / 4

        P = np.eye(n)
        P[i, i] = P[j, j] = np.cos(angle)
        P[i, j] = -np.sin(angle)
        P[j, i] = np.sin(angle)

        A = P.T @ A @ P
        Q = Q @ P

        iterations += 1

    eigen_values = np.diag(A)
    eigen_vectors = [Q[0:n, i] for i in range(n)]

    return (eigen_values, eigen_vectors)



In [5]:
n = 5
A = make_matrix(n)
print(f"Матрица имеет вид:\n{A}")

Матрица имеет вид:
[[0.17 0.32 0.31 0.75 0.33]
 [0.32 0.17 0.58 0.46 0.84]
 [0.31 0.58 0.17 0.83 0.61]
 [0.75 0.46 0.83 0.41 0.28]
 [0.33 0.84 0.61 0.28 0.55]]


In [8]:
eigen_value_s, eigen_vector_s = scalar_product_method(A)
print(f"Максимальное по модулю собс. значение: {eigen_value_s}")
print(f"Собственный вектор: {eigen_vector_s}")
print(f"Вектор невязки: {A @ eigen_vector_s - eigen_value_s * eigen_vector_s}")

Максимальное по модулю собс. значение: 2.4465280845189463
Собственный вектор: [0.35 0.44 0.46 0.49 0.48]
Вектор невязки: [-8.71e-07 -5.74e-08 -1.27e-06  1.36e-06  5.42e-07]


In [9]:
eigen_value_p, eigen_vector_p = power_method(A)
print(f"Максимальное по модулю собс. значение: {eigen_value_p}")
print(f"Собственный вектор: {eigen_vector_p}")
print(f"Вектор невязки: {A @ eigen_vector_p - eigen_value_p * eigen_vector_p}")

Максимальное по модулю собс. значение: 2.4465280845143083
Собственный вектор: [0.35 0.44 0.46 0.49 0.48]
Вектор невязки: [ 9.98e-12  1.74e-12  1.42e-11 -1.57e-11 -6.90e-12]


In [10]:
B = A - eigen_value_s * np.eye(n)
b_eigen_value, b_eigen_vector = scalar_product_method(B)
min_eigen_value = b_eigen_value + eigen_value_s
print(f"Минимальное собственное значение матрицы: {min_eigen_value}")

Минимальное собственное значение матрицы: -0.7473627663023965


In [11]:
eigen_values, eigen_vectors = jacobi_method(A)
print("Собственные значения, вектора и вектора невязки:")
for i, v in enumerate(eigen_values):
    vec = eigen_vectors[i]
    print(f"{i}\t{v}")
    print(f"\t{vec}")
    print(f"\t{A @ vec - v * vec}")

Собственные значения, вектора и вектора невязки:
0	-0.1785350248394256
	[ 0.73  0.01 -0.61 -0.2   0.25]
	[-2.49e-07  3.35e-06 -1.09e-06  1.07e-07 -1.97e-06]
1	-0.5262404554538299
	[-0.06  0.83 -0.27  0.02 -0.49]
	[ 2.94e-06  3.47e-08 -2.46e-06 -8.10e-07  1.01e-06]
2	-0.747362766934942
	[ 0.41  0.07  0.58 -0.64 -0.28]
	[-7.67e-07 -1.09e-07 -5.01e-07 -1.01e-06  1.36e-07]
3	0.46971554582447916
	[ 0.42 -0.34  0.06  0.56 -0.63]
	[-3.75e-07 -4.40e-08 -5.38e-07  6.48e-07  2.99e-07]
4	2.446528084514007
	[0.35 0.44 0.46 0.49 0.48]
	[-3.76e-07 -8.81e-08 -5.67e-07  6.59e-07  2.41e-07]
